<a href="https://colab.research.google.com/github/Droid008/Introduction-to-the-processing-of-a-natural-language/blob/Lesson-%238/hw_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv
Saving val.csv to val.csv


In [4]:
import pandas as pd
pd.set_option('max_colwidth', 120)
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

In [5]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_val = pd.read_csv("val.csv")

In [6]:
df_train.head(5)

,id,text,class
0,0,"@alisachachka не уезжаааааааай. :(❤ я тоже не хочу, чтобы ты уезжала.",0
1,1,"RT @GalyginVadim: Ребята и девчата!\nВсе в кино!!! ""Вот Это Любовь!""\nСегодня! Завтра! И потом!)))))\n#вотэтолюбовь",1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретвит((((( RT,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Запретный плод. :),1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса не боится мороза и .......)),1


In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [9]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [10]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [11]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [12]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    output_dim=128, 
                    input_length=training_length, 
                    trainable=True,
                    mask_zero=True)
         )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )

In [13]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 148s 459ms/step - loss: 0.5412 - accuracy: 0.7126 - val_loss: 0.4834 - val_accuracy: 0.7612
Epoch 2/10
319/319 [==============================] - 142s 446ms/step - loss: 0.2677 - accuracy: 0.8911 - val_loss: 0.5406 - val_accuracy: 0.7488


In [14]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 33ms/step - loss: 0.5981 - accuracy: 0.7320


Test score: 0.5981391668319702
Test accuracy: 0.7319578528404236


In [15]:
compare_result = pd.DataFrame(columns=['Model', 'Score', 'Accuracy'])
compare_result.loc[0] = ['CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.598139,0.731958


# RNN

In [16]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
         )
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )

In [17]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 50s 150ms/step - loss: 0.5586 - accuracy: 0.7007 - val_loss: 0.4911 - val_accuracy: 0.7565
Epoch 2/10
319/319 [==============================] - 47s 147ms/step - loss: 0.2785 - accuracy: 0.8875 - val_loss: 0.5695 - val_accuracy: 0.7404


In [18]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 20ms/step - loss: 0.5858 - accuracy: 0.7360


Test score: 0.5858233571052551
Test accuracy: 0.7359696626663208


In [19]:
compare_result.loc[1] = ['RNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.598139,0.731958
1,RNN,0.585823,0.735970


## LSTM

In [20]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy']
              )

In [21]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train,
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 91s 275ms/step - loss: 0.5539 - accuracy: 0.7062 - val_loss: 0.4929 - val_accuracy: 0.7564
Epoch 2/10
319/319 [==============================] - 86s 270ms/step - loss: 0.3271 - accuracy: 0.8646 - val_loss: 0.5387 - val_accuracy: 0.7504


In [22]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 39ms/step - loss: 0.5766 - accuracy: 0.7387


Test score: 0.5766400694847107
Test accuracy: 0.7387030124664307


In [23]:
compare_result.loc[2] = ['LSTM', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.598139,0.731958
1,RNN,0.585823,0.735970
2,LSTM,0.576640,0.738703


## GRU

In [24]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    trainable=True,
                    mask_zero=True)
         )
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
              )

In [25]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, y_train,batch_size=512,epochs=10,verbose=1,validation_split=0.1,callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 79s 240ms/step - loss: 0.5499 - accuracy: 0.7137 - val_loss: 0.4939 - val_accuracy: 0.7553
Epoch 2/10
319/319 [==============================] - 75s 236ms/step - loss: 0.3178 - accuracy: 0.8686 - val_loss: 0.5322 - val_accuracy: 0.7429


In [26]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 33ms/step - loss: 0.5816 - accuracy: 0.7352


Test score: 0.5816155672073364
Test accuracy: 0.735220193862915


In [27]:
compare_result.loc[3] = ['GRU', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.598139,0.731958
1,RNN,0.585823,0.735970
2,LSTM,0.576640,0.738703
3,GRU,0.581616,0.735220


In [28]:
compare_result

,Model,Score,Accuracy
0,CNN,0.598139,0.731958
1,RNN,0.585823,0.735970
2,LSTM,0.576640,0.738703
3,GRU,0.581616,0.735220


## CNN-SimpleRNN


In [29]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 58s 178ms/step - loss: 0.5575 - accuracy: 0.6993 - val_loss: 0.4887 - val_accuracy: 0.7575
Epoch 2/10
319/319 [==============================] - 57s 178ms/step - loss: 0.2763 - accuracy: 0.8870 - val_loss: 0.5505 - val_accuracy: 0.7444


In [30]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 28ms/step - loss: 0.5779 - accuracy: 0.7365


Test score: 0.57786625623703
Test accuracy: 0.7364546060562134


In [31]:
compare_result.loc[4] = ['CNN-SimpleRNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.598139,0.731958
1,RNN,0.585823,0.735970
2,LSTM,0.576640,0.738703
3,GRU,0.581616,0.735220
4,CNN-SimpleRNN,0.577866,0.736455


## CNN - LSTM

In [32]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train,
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 109s 333ms/step - loss: 0.5510 - accuracy: 0.7072 - val_loss: 0.4890 - val_accuracy: 0.7580
Epoch 2/10
319/319 [==============================] - 107s 334ms/step - loss: 0.2970 - accuracy: 0.8767 - val_loss: 0.5350 - val_accuracy: 0.7485


In [33]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 3s 58ms/step - loss: 0.5719 - accuracy: 0.7402


Test score: 0.5718802809715271
Test accuracy: 0.7402018904685974


In [34]:
compare_result.loc[5] = ['CNN - LSTM', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.598139,0.731958
1,RNN,0.585823,0.735970
2,LSTM,0.576640,0.738703
3,GRU,0.581616,0.735220
4,CNN-SimpleRNN,0.577866,0.736455
5,CNN - LSTM,0.571880,0.740202


## CNN-GRU

In [35]:
model = Sequential()
model.add(Embedding(input_dim=word_count,
                    input_length=training_length,
                    output_dim=30,
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 100s 303ms/step - loss: 0.5496 - accuracy: 0.7081 - val_loss: 0.4980 - val_accuracy: 0.7538
Epoch 2/10
319/319 [==============================] - 95s 298ms/step - loss: 0.2924 - accuracy: 0.8795 - val_loss: 0.5736 - val_accuracy: 0.7425


In [36]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 46ms/step - loss: 0.6467 - accuracy: 0.7310


Test score: 0.6466670036315918
Test accuracy: 0.7309879660606384


In [37]:
compare_result.loc[6] = ['CNN-GRU', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.598139,0.731958
1,RNN,0.585823,0.735970
2,LSTM,0.576640,0.738703
3,GRU,0.581616,0.735220
4,CNN-SimpleRNN,0.577866,0.736455
5,CNN - LSTM,0.571880,0.740202
6,CNN-GRU,0.646667,0.730988


In [38]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(SimpleRNN(64, recurrent_dropout=0.2, return_sequences="True"))
model.add(Conv1D(64, 3, activation="linear"))
model.add(Conv1D(64, 1, activation="linear")) 
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(1, activation="sigmoid"))      


model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 70s 213ms/step - loss: 0.5570 - accuracy: 0.7008 - val_loss: 0.4960 - val_accuracy: 0.7545
Epoch 2/10
319/319 [==============================] - 68s 212ms/step - loss: 0.3121 - accuracy: 0.8705 - val_loss: 0.5538 - val_accuracy: 0.7421


In [39]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 30ms/step - loss: 0.5698 - accuracy: 0.7343


Test score: 0.5697512030601501
Test accuracy: 0.7342503070831299


In [40]:
compare_result.loc[7] = ['SimpleRNN-CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.598139,0.731958
1,RNN,0.585823,0.735970
2,LSTM,0.576640,0.738703
3,GRU,0.581616,0.735220
4,CNN-SimpleRNN,0.577866,0.736455
5,CNN - LSTM,0.571880,0.740202
6,CNN-GRU,0.646667,0.730988
7,SimpleRNN-CNN,0.569751,0.734250


## LSTM - CNN

In [41]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 90s 273ms/step - loss: 0.5519 - accuracy: 0.7092 - val_loss: 0.4935 - val_accuracy: 0.7547
Epoch 2/10
319/319 [==============================] - 86s 269ms/step - loss: 0.3258 - accuracy: 0.8644 - val_loss: 0.5351 - val_accuracy: 0.7483


In [42]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 38ms/step - loss: 0.5767 - accuracy: 0.7379


Test score: 0.5767333507537842
Test accuracy: 0.7378653883934021


In [43]:
compare_result.loc[8] = ['LSTM - CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.598139,0.731958
1,RNN,0.585823,0.735970
2,LSTM,0.576640,0.738703
3,GRU,0.581616,0.735220
4,CNN-SimpleRNN,0.577866,0.736455
5,CNN - LSTM,0.571880,0.740202
6,CNN-GRU,0.646667,0.730988
7,SimpleRNN-CNN,0.569751,0.734250
8,LSTM - CNN,0.576733,0.737865


## GRU-CNN

In [45]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0)) 
model.add(GRU(64,  recurrent_dropout=0.2, return_sequences="True")) 
model.add(Conv1D(64, 3, activation="linear"))
model.add(Conv1D(64, 1, activation="linear")) 
model.add(Flatten())                      
model.add(Dropout(0.5)) 
model.add(Dense(1, activation="sigmoid"))      


model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 99s 289ms/step - loss: 0.5508 - accuracy: 0.7048 - val_loss: 0.4945 - val_accuracy: 0.7531
Epoch 2/10
319/319 [==============================] - 91s 286ms/step - loss: 0.3053 - accuracy: 0.8731 - val_loss: 0.5435 - val_accuracy: 0.7483


In [46]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 43ms/step - loss: 0.5654 - accuracy: 0.7371


Test score: 0.5653643012046814
Test accuracy: 0.7371159195899963


In [47]:
compare_result.loc[9] = ['GRU - CNN', score[0], score[1]]
compare_result

,Model,Score,Accuracy
0,CNN,0.598139,0.731958
1,RNN,0.585823,0.735970
2,LSTM,0.576640,0.738703
3,GRU,0.581616,0.735220
4,CNN-SimpleRNN,0.577866,0.736455
5,CNN - LSTM,0.571880,0.740202
6,CNN-GRU,0.646667,0.730988
7,SimpleRNN-CNN,0.569751,0.734250
8,LSTM - CNN,0.576733,0.737865
9,GRU - CNN,0.565364,0.737116
